
<div style="background-color:#f2f2f2;padding:20px;text-align:center;">
  <h1 style="color:#0b61a4;margin:0;">Clase 4 — Notebook de Limpieza y Validación de DataFrames</h1>
  <p style="color:#0b61a4;margin:0;">Pipeline pedagógico para detectar, normalizar y reportar problemas en datasets CSV</p>
</div>



<div style="background-color:#f9f9f9;padding:12px;">
<p style="color:#0b61a4;">
Este notebook está diseñado con propósito educativo: muestra un flujo completo para cargar tres CSV (ventas, clientes, marketing),
detectar problemas (duplicados, nulos, tipos, outliers), aplicar reglas de limpieza parametrizadas y generar reportes 'antes' y 'después'.
</p>

<p style="color:#0b61a4;">
Se han unificado los nombres de variables al español y en formato <code>snake_case</code>.<br>
- <code>issues_antes</code> → <code>problemas_antes</code><br>
- <code>issues_despues</code> → <code>problemas_despues</code><br>
- <code>chequeos_columnas_despues</code> → <code>chequeos_despues</code>
</p>

<p style="color:#0b61a4;">
Las celdas contienen documentación explicativa y las funciones están completamente comentadas en español para uso docente.
</p>
</div>


In [1]:

# Importar librerías necesarias
import os
import sys
import zipfile
import shutil
from datetime import datetime
from typing import Tuple, Dict, Any

import pandas as pd
import numpy as np

# Para métricas estadísticas simples
from scipy import stats

# Configuración de rutas por defecto (local)
ruta_base = os.path.abspath(".")
ruta_entrada = os.path.join(ruta_base, "datasets_entrada")
ruta_salida = os.path.join(ruta_base, "datasets_salida")
directorio_reportes = os.path.join(ruta_salida, "reportes")

# Asegurar que existen las carpetas necesarias
os.makedirs(ruta_entrada, exist_ok=True)
os.makedirs(ruta_salida, exist_ok=True)
os.makedirs(directorio_reportes, exist_ok=True)

# Mostrar rutas configuradas
print("Ruta de entrada:", ruta_entrada)
print("Ruta de salida:", ruta_salida)
print("Directorio de reportes:", directorio_reportes)


Ruta de entrada: D:\Desktop\Domingo\df_caba_y_jupyter\datasets_entrada
Ruta de salida: D:\Desktop\Domingo\df_caba_y_jupyter\datasets_salida
Directorio de reportes: D:\Desktop\Domingo\df_caba_y_jupyter\datasets_salida\reportes



<h3 style="color:#0b61a4;">Funciones auxiliares de normalización y utilidad</h3>
<p style="color:#0b61a4;">A continuación definimos funciones reutilizables que ayudan a normalizar nombres, limpiar textos, convertir tipos y detectar duplicados.</p>


In [2]:

def normalizar_nombre_columna(nombre: str) -> str:
    """
    Normaliza un nombre de columna a snake_case, sin tildes y en minúsculas.
    Ejemplo: 'Fecha Venta' -> 'fecha_venta'
    """
    import re
    # Remover tildes básicas
    reemplazos = str.maketrans("ÁÉÍÓÚáéíóúÑñ", "AEIOUaeiouNn")
    nombre = nombre.translate(reemplazos)
    # Reemplazar separadores por underscore
    nombre = re.sub(r'[^\w]+', '_', nombre)
    nombre = nombre.strip('_').lower()
    return nombre

def normalizar_nombres_columnas(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aplica normalización de nombres de columnas a todo el DataFrame.
    """
    df = df.copy()
    df.columns = [normalizar_nombre_columna(c) for c in df.columns]
    return df

def convertir_a_numerico(col: pd.Series, remove_non_digits: bool = False, as_int: bool = False) -> pd.Series:
    """
    Intenta convertir una columna a tipo numérico.
    - remove_non_digits: elimina caracteres no numéricos (útil para montos con símbolos)
    - as_int: si True, convierte a int cuando sea posible (rellena NaN con 0 antes)
    """
    s = col.astype(str).copy()
    if remove_non_digits:
        s = s.str.replace(r'[^0-9\.\-]', '', regex=True)
    # Reemplazar valores vacíos o solo '-' por NaN
    s = s.replace({'': np.nan, 'nan': np.nan, '-': np.nan})
    # Convertir a float
    s = pd.to_numeric(s, errors='coerce')
    if as_int:
        # Rellenar NaN con 0 temporalmente para conversion segura si se desea
        s = s.fillna(0).round().astype('Int64')
    return s

def intentar_convertir_fecha(col: pd.Series, formatos: list = None, dayfirst: bool = True) -> pd.Series:
    """
    Intenta convertir una columna a datetime usando varios formatos.
    Si 'formatos' es None, utiliza infer_datetime_format.
    """
    if formatos:
        for fmt in formatos:
            try:
                parsed = pd.to_datetime(col, format=fmt, dayfirst=dayfirst, errors='coerce')
                # Si se parseó con éxito muchos valores, adoptamos este formato
                if parsed.notna().sum() > 0:
                    return parsed
            except Exception:
                continue
    # Fallback: inferir
    return pd.to_datetime(col, errors='coerce', dayfirst=dayfirst, infer_datetime_format=True)



<h3 style="color:#0b61a4;">Función: detectar_problemas_dataframe</h3>
<p style="color:#0b61a4;">Detecta problemas comunes en un DataFrame: duplicados, nulos por columna, tipos inconsistentes y outliers básicos (IQR).</p>


In [3]:

def detectar_problemas_dataframe(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Analiza un DataFrame y devuelve:
    - resumen: DataFrame resumen con métricas por columna
    - chequeos: DataFrame con detalles por columna (tipos detectados, nulos, únicos)
    - problemas: DataFrame con filas que contienen al menos un problema detectado (duplicados o nulos críticos)

    NOTAS:
    - Esta función es didáctica; se puede ampliar para detectar outliers avanzados o reglas de negocio.
    """
    df = df.copy()
    resumen = []
    chequeos = []
    n = len(df)

    # Detección de duplicados (filas completas)
    duplicados = df.duplicated(keep=False)

    for col in df.columns:
        serie = df[col]
        n_nulos = serie.isna().sum()
        n_unicos = serie.nunique(dropna=True)
        tipo_inferido = serie.dropna().map(type).value_counts().index.tolist()
        tipo_str = ','.join([t.__name__ for t in tipo_inferido]) if tipo_inferido else 'empty'
        # Detectar outliers simples por IQR si la columna es numérica
        outliers = None
        if pd.api.types.is_numeric_dtype(serie):
            q1 = serie.quantile(0.25)
            q3 = serie.quantile(0.75)
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
            outliers = int(((serie < lower) | (serie > upper)).sum())
        else:
            outliers = np.nan

        resumen.append({
            'columna': col,
            'tipo_aproximado': tipo_str,
            'nulos': int(n_nulos),
            'unicos': int(n_unicos),
            'outliers_estimados': outliers
        })
        chequeos.append({
            'columna': col,
            'tipo_pandas': str(serie.dtype),
            'ejemplo_valores': ','.join(map(str, serie.dropna().unique()[:5])),
            'porcentaje_nulos': float(n_nulos) / n if n>0 else 0.0
        })
    resumen_df = pd.DataFrame(resumen)
    chequeos_df = pd.DataFrame(chequeos)

    # Filas con problemas: duplicados o filas con todos los valores NaN
    filas_problema_mask = duplicados | df.isna().all(axis=1)
    problemas_df = df[filas_problema_mask].copy()
    # Si no hay filas problema, devolver empty DataFrame con columnas del origen + motivo
    if problemas_df.empty:
        problemas_df = pd.DataFrame(columns=list(df.columns) + ['motivo'])
    else:
        # Añadir motivo
        motivos = []
        for idx, row in problemas_df.iterrows():
            m = []
            if duplicated := df.loc[[idx]].duplicated(keep=False).any():
                m.append('duplicado')
            if row.isna().all():
                m.append('fila_vacia')
            motivos.append(';'.join(m) if m else 'otro')
        problemas_df['motivo'] = motivos

    return resumen_df, chequeos_df, problemas_df



<h3 style="color:#0b61a4;">Función: aplicar_reglas_limpieza</h3>
<p style="color:#0b61a4;">Aplica un conjunto de reglas parametrizadas a un DataFrame para normalizar tipos y valores.</p>


In [4]:

def aplicar_reglas_limpieza(df: pd.DataFrame, reglas: Dict[str, Any]) -> pd.DataFrame:
    """
    Aplica reglas sobre columnas especificadas en 'reglas'.
    'reglas' es un dict donde la clave es el nombre de la columna (ya normalizado)
    y el valor indica la operación a realizar:
      - 'title', 'lower' -> operaciones de cadena
      - ('numeric', params) -> convertir a numérico (params es dict con opciones)
      - ('date', params) -> convertir a datetime (params puede contener 'formats' list)
    """
    df = df.copy()
    for columna, regla in reglas.items():
        if columna not in df.columns:
            # No existe la columna; doc: se saltea silenciosamente pero se informa
            print(f"Aviso: la columna '{columna}' no existe en el DataFrame. Se omite.")
            continue
        if regla == 'title':
            df[columna] = df[columna].astype(str).str.strip().str.title().replace({'nan': pd.NA})
        elif regla == 'lower':
            df[columna] = df[columna].astype(str).str.strip().str.lower().replace({'nan': pd.NA})
        elif isinstance(regla, tuple) and regla[0] == 'numeric':
            params = regla[1] if len(regla) > 1 else {}
            remove_non = params.get('remove_non_digits', False)
            as_int = params.get('as_int', False)
            df[columna] = convertir_a_numerico(df[columna], remove_non_digits=remove_non, as_int=as_int)
        elif isinstance(regla, tuple) and regla[0] == 'date':
            params = regla[1] if len(regla) > 1 else {}
            formatos = params.get('formats', None)
            dayfirst = params.get('dayfirst', True)
            df[columna] = intentar_convertir_fecha(df[columna], formatos=formatos, dayfirst=dayfirst)
        else:
            print(f"Aviso: regla desconocida para columna '{columna}': {regla}")
    return df



<h3 style="color:#0b61a4;">Función: procesar_carpeta</h3>
<p style="color:#0b61a4;">Procesa todos los CSV en una carpeta de entrada, aplica limpieza, crea reportes 'antes' y 'después' y guarda resultados.</p>


In [5]:

def procesar_carpeta(ruta_entrada: str, ruta_salida: str, reglas_columnas: Dict[str, Any]=None):
    """
    Recorre todos los archivos CSV en 'ruta_entrada', aplica detección de problemas,
    limpieza con 'reglas_columnas' y guarda:
      - reportes antes y después en ruta_salida/reportes/
      - archivos limpios en ruta_salida/csv_limpios/
      - un ZIP con todos los reportes

    Variables de salida estandarizadas en español:
      - problemas_antes
      - resumen_antes
      - chequeos_antes
      - problemas_despues
      - resumen_despues
      - chequeos_despues
    """
    os.makedirs(ruta_salida, exist_ok=True)
    carpeta_reportes = os.path.join(ruta_salida, "reportes")
    carpeta_csv_limpios = os.path.join(ruta_salida, "csv_limpios")
    os.makedirs(carpeta_reportes, exist_ok=True)
    os.makedirs(carpeta_csv_limpios, exist_ok=True)

    archivos = [f for f in os.listdir(ruta_entrada) if f.lower().endswith('.csv')]
    if not archivos:
        print("No se encontraron archivos CSV en la carpeta de entrada:", ruta_entrada)
        return

    for archivo in archivos:
        nombre_archivo = os.path.splitext(archivo)[0]
        ruta_archivo = os.path.join(ruta_entrada, archivo)
        print(f"\nProcesando: {ruta_archivo}")

        # Cargar CSV
        df = pd.read_csv(ruta_archivo, encoding='utf-8', low_memory=False)
        # Normalizar nombres de columnas
        df = normalizar_nombres_columnas(df)

        # Detectar problemas antes de limpieza
        resumen_antes, chequeos_antes, problemas_antes = detectar_problemas_dataframe(df)

        # Guardar reporte 'antes'
        ruta_reporte_antes = os.path.join(carpeta_reportes, f"reporte_antes_limpieza_{nombre_archivo}.csv")
        resumen_antes.to_csv(ruta_reporte_antes.replace('.csv','_resumen.csv'), index=False, encoding='utf-8')
        chequeos_antes.to_csv(ruta_reporte_antes.replace('.csv','_chequeos.csv'), index=False, encoding='utf-8')
        problemas_antes.to_csv(ruta_reporte_antes.replace('.csv','_problemas.csv'), index=False, encoding='utf-8')
        print("Reportes 'antes' guardados en:", carpeta_reportes)

        # Aplicar reglas de limpieza si existen
        if reglas_columnas:
            df_limpio = aplicar_reglas_limpieza(df, reglas_columnas)
        else:
            df_limpio = df.copy()

        # Detectar problemas despues de limpieza
        resumen_despues, chequeos_despues, problemas_despues = detectar_problemas_dataframe(df_limpio)

        # Guardar reporte 'despues'
        ruta_reporte_despues = os.path.join(carpeta_reportes, f"reporte_despues_limpieza_{nombre_archivo}.csv")
        resumen_despues.to_csv(ruta_reporte_despues.replace('.csv','_resumen.csv'), index=False, encoding='utf-8')
        chequeos_despues.to_csv(ruta_reporte_despues.replace('.csv','_chequeos.csv'), index=False, encoding='utf-8')
        problemas_despues.to_csv(ruta_reporte_despues.replace('.csv','_problemas.csv'), index=False, encoding='utf-8')
        print("Reportes 'despues' guardados en:", carpeta_reportes)

        # Guardar CSV limpio
        ruta_csv_limpio = os.path.join(carpeta_csv_limpios, f"{nombre_archivo}_limpio.csv")
        df_limpio.to_csv(ruta_csv_limpio, index=False, encoding='utf-8')
        print("CSV limpio guardado en:", ruta_csv_limpios)

    # Crear ZIP de reportes
    zip_path = os.path.join(ruta_salida, "reportes_dataset.zip")
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(carpeta_reportes):
            for f in files:
                zf.write(os.path.join(root, f), arcname=os.path.join(os.path.basename(root), f))
    print("ZIP de reportes creado en:", zip_path)



<h3 style="color:#0b61a4;">Ejecución principal: función <code>menu()</code></h3>
<p style="color:#0b61a4;">La función <code>menu()</code> prepara archivos de ejemplo si no existen y ejecuta <code>procesar_carpeta</code>.</p>


In [6]:

def crear_datasets_ejemplo(ruta_entrada: str):
    """
    Crea 3 CSV de ejemplo si no existen en la carpeta de entrada.
    Estos archivos representan: ventas, clientes y marketing.
    """
    # Ejemplo simple de ventas
    df_ventas = pd.DataFrame({
        'id_venta': [1,2,3,3],
        'fecha_venta': ['01/01/2021','02/01/2021','03/01/2021', '03/01/2021'],
        'producto': ['camisa','pantalon','camisa','camisa'],
        'cantidad': ['1','2','1','1'],
        'precio': ['100.0','200.0','100','100']
    })
    df_clientes = pd.DataFrame({
        'id_cliente': [10,11,12],
        'nombre': ['Juan Pérez','María López','Carlos Gómez'],
        'ciudad': ['Buenos Aires','Cordoba','Rosario']
    })
    df_marketing = pd.DataFrame({
        'campana': ['fb_enero','google_febrero'],
        'gasto': ['1000','1500'],
        'canal': ['Facebook','Google']
    })
    # Guardar si no existen
    df_ventas.to_csv(os.path.join(ruta_entrada, "ventas.csv"), index=False, encoding='utf-8')
    df_clientes.to_csv(os.path.join(ruta_entrada, "clientes.csv"), index=False, encoding='utf-8')
    df_marketing.to_csv(os.path.join(ruta_entrada, "marketing.csv"), index=False, encoding='utf-8')
    print("Datasets de ejemplo creados en:", ruta_entrada)

def menu():
    """
    Función principal de ejecución del pipeline de limpieza.

    OBJETIVO EDUCATIVO:
    Muestra cómo integrar todas las piezas en un flujo automatizado,
    desde la carga de datos hasta la generación de reportes finales.
    """
    print("Iniciando pipeline de limpieza de datos...")
    print(f"Directorio base: {ruta_base}")
    print(f"Ruta entrada: {ruta_entrada}")
    print(f"Ruta salida: {ruta_salida}")

    # Si no hay CSV, crear ejemplos
    csvs = [f for f in os.listdir(ruta_entrada) if f.lower().endswith('.csv')]
    if not csvs:
        crear_datasets_ejemplo(ruta_entrada)

    # Mostrar preview de los datos originales (solo primeros archivos)
    archivos = [f for f in os.listdir(ruta_entrada) if f.lower().endswith('.csv')]
    for archivo in archivos:
        ruta = os.path.join(ruta_entrada, archivo)
        df_temp = pd.read_csv(ruta, encoding='utf-8', low_memory=False)
        df_temp = normalizar_nombres_columnas(df_temp)
        print("\n--- PREVIEW:", archivo, "---")
        print(df_temp.head(3))
        print("Dimensiones:", df_temp.shape)

    # Definir reglas de limpieza de ejemplo (puede personalizarse)
    reglas_columnas = {
        'nombre': 'title',
        'ciudad': 'title',
        'producto': 'title',
        'categoria': 'title',
        'canal': 'lower',
        'precio': ('numeric', {'remove_non_digits': False}),
        'cantidad': ('numeric', {'as_int': True}),
        'ingresos': ('numeric', {}),
        'costo': ('numeric', {}),
        'fecha_venta': ('date', {'formats': ['%d/%m/%Y'], 'dayfirst': True}),
        'fecha_inicio': ('date', {'formats': ['%d/%m/%Y'], 'dayfirst': True}),
        'fecha_fin': ('date', {'formats': ['%d/%m/%Y'], 'dayfirst': True})
    }

    # Ejecutar procesamiento
    procesar_carpeta(ruta_entrada, ruta_salida, reglas_columnas=reglas_columnas)
    print("Pipeline completado.")

# Ejecutar menu
menu()


Iniciando pipeline de limpieza de datos...
Directorio base: D:\Desktop\Domingo\df_caba_y_jupyter
Ruta entrada: D:\Desktop\Domingo\df_caba_y_jupyter\datasets_entrada
Ruta salida: D:\Desktop\Domingo\df_caba_y_jupyter\datasets_salida

--- PREVIEW: clientes2.csv ---
   id_cliente               nombre  edad         ciudad  ingresos
0           1      Aloysia Screase    44  Mar del Plata  42294.68
1           2  Kristina Scaplehorn    25        Posadas  24735.04
2           3       Filip Castagne    50    Resistencia  35744.85
Dimensiones: (567, 5)

--- PREVIEW: marketing2.csv ---
   id_campanha         producto  canal  costo fecha_inicio   fecha_fin
0           74  Adorno de pared     TV   4.81   20/03/2024  03/05/2024
1           12           Tablet   RRSS   3.40   26/03/2024  13/05/2024
2           32  Lámpara de mesa  Email   5.54   28/03/2024  20/04/2024
Dimensiones: (90, 6)

--- PREVIEW: ventas2.csv ---
   id_venta           producto   precio  cantidad fecha_venta  \
0       792  Cuadr

NameError: name 'ruta_csv_limpios' is not defined


<p style="color:#0b61a4;">El notebook ha sido generado con funciones completas. Si deseas, puedes descargar el archivo .ipynb generado en /mnt/data al ejecutar la celda que crea y guarda el notebook.</p>
